**Connect google drive**

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Wed Nov 25 08:57:49 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 20201014_162728.mp4  'images + .txt'   sans_zip		     yolov_3
 20201015_165447.mp4  'My Drive'        yolo_custom_model_Training   zippe
'Colab Notebooks'      projects         yolov3


**1) Clone, configure & compile Darknet**



In [3]:
# Clone
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 14479 (delta 7), reused 13 (delta 7), pack-reused 14460
Receiving objects: 100% (14479/14479), 13.22 MiB | 22.08 MiB/s, done.
Resolving deltas: 100% (9842/9842), done.


In [4]:
# Configure
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [5]:
# Compile
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

**2) Configure yolov3.cfg file**

In [6]:
# Make a copy of yolov3.cfg
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [7]:
# Change lines in yolov3.cfg file
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 2000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

**3) Create .names and .data files**

In [8]:
!echo -e 'Citrouille' > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data

**4) Save yolov3_training.cfg and obj.names files in Google drive**

In [9]:
!cp cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_testing.cfg
!cp data/obj.names /mydrive/yolov3/classes.txt

**5) Create a folder and unzip image dataset**

In [10]:
!mkdir data/obj
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/00324600bac57e93.jpg  
  inflating: data/obj/00324600bac57e93.txt  
  inflating: data/obj/004d534a2552a90d.jpg  
  inflating: data/obj/004d534a2552a90d.txt  
  inflating: data/obj/004dd6d3a863262e.jpg  
  inflating: data/obj/004dd6d3a863262e.txt  
  inflating: data/obj/0082df1b488dcabd.jpg  
  inflating: data/obj/0082df1b488dcabd.txt  
  inflating: data/obj/0085a95252820f7e.jpg  
  inflating: data/obj/0085a95252820f7e.txt  
  inflating: data/obj/00c2ee786e4695c2.jpg  
  inflating: data/obj/00c2ee786e4695c2.txt  
  inflating: data/obj/00f5e7e442cad11f.jpg  
  inflating: data/obj/00f5e7e442cad11f.txt  
  inflating: data/obj/0116e32f26c32b9c.jpg  
  inflating: data/obj/0116e32f26c32b9c.txt  
  inflating: data/obj/016c0fb0708597a7.jpg  
  inflating: data/obj/016c0fb0708597a7.txt  
  inflating: data/obj/018b641228674ec1.jpg  
  inflating: data/obj/018b641228674ec1.txt  
  inflating: data/obj/01a51e514ae1c3fa.jpg  
  inflating: data/

**6) Create train.txt file**

In [11]:
import glob
images_list = glob.glob("data/obj/*.jpg")
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))

**7) Download pre-trained weights for the convolutional layers file**

In [12]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-11-25 09:07:43--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   173KB/s    in 19m 8s  

2020-11-25 09:26:52 (138 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**8) Start training**

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show
# Uncomment below and comment above to re-start your training from last saved weights
#!./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_training_last.weights -dont_show

Streaming output truncated to the last 5000 lines.
 1884: 0.636152, 0.623017 avg loss, 0.001000 rate, 5.597852 seconds, 120576 images, 0.391752 hours left
Loaded: 0.000041 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.798979), count: 4, total_loss = 0.470594 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.861086), count: 2, total_loss = 0.043792 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, total_loss = 0.000025 
 total_bbox = 393248, rewritten_bbox = 0.539354 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.796079), count: 6, total_loss = 0.612315 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.681186), count: 1, total_loss = 0.179408 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.542322), count: 1, total_loss = 0.380180 
 total_bbox = 393256, r